This notebook is part of a project: 
Characteristics Corona patients DS

the project's target :Build a clean and detailed database as detailed as possible on the characteristics of Corona patients from around the world

Purpose of the notebook:
Build a patient-level database
from different information sets
by cleaning the heating and connecting the arrays

In [ ]:
import nltk
import time 
import pandas as pd 
import  numpy as np
from collections import Counter
import time 
from control import *
from clean import *
from clean_time import *
from clean_age import *
from text_analysis import *
from python_expansion import *
from jengineer import *

In [ ]:
Control.v()
Clean.v()
CTime.v()
Cage.v()
TextAnalysis.v()
Pexpansion.v()
JEngineer.v()

In [ ]:
%%time
start_all = time.time()
# import all datasets
canada_cases = pd.read_csv("/kaggle/input/coronaviruscovid19-canada/cases.csv")
canada_dead = pd.read_csv("/kaggle/input/coronaviruscovid19-canada/mortality.csv")

india_data  = pd.read_csv("/kaggle/input/covid19india/data.csv")
india_wiki = pd.read_csv("/kaggle/input/covid19-coronavirus-patient-list-karnataka-india/Corona Covid19 Karnataka.csv")

france = pd.read_csv("/kaggle/input/coronavirus-france-dataset/patient.csv")
tunisia = pd.read_csv("/kaggle/input/coronavirus-tunisia/Coronavirus_Tunisia.csv")
japan = pd.read_csv("/kaggle/input/close-contact-status-of-corona-in-japan/COVID-19_Japan_Mar_07th_2020.csv")
indonesia = pd.read_csv("/kaggle/input/indonesia-coronavirus-cases/patient.csv")
korea = pd.read_csv("/kaggle/input/coronavirusdataset/PatientInfo.csv")
new_zealand = pd.read_csv("/kaggle/input/covid19-nz/COVID19-NZ-known-cases (updated 23-3.2020).csv")
guatemala = pd.read_csv("/kaggle/input/covid19-guatemala/patients.csv")
singapore = pd.read_csv("/kaggle/input/singapores-covid19-cases/singapore_covid-19_sg_cases_updated-2020-04-14.csv")
vietnam = pd.read_csv("/kaggle/input/vietnam-covid19-patient-dataset/Vietnam_COVID-19_patient_data_May10_2020.csv")
colombia = pd.read_csv("/kaggle/input/covid19co/covid19co.csv")
mexico = pd.read_csv("/kaggle/input/covid19-mx/covid-19_general_MX.csv")
kerla = pd.read_csv("/kaggle/input/covid19inkerala/PatientData.csv")
philippines  = pd.read_excel("/kaggle/input/covid19-philippines-edges/COVID-19 Philippines.xlsx", sheet_name="Cases")
toronto = pd.read_csv(r"/kaggle/input/toronto-covid19-cases/COVID19 cases Toronto.csv")
hong_kong =  pd.read_csv(r"/kaggle/input/covid-19-hong-kong-cases/covid19case_full_17_08.csv")
world = pd.read_csv(r"/kaggle//input/covid19-re/06122020latestdata.csv")
usa =  pd.read_csv(r"/kaggle/input/covid19-case-surveillance-public-use-dataset/COVID-19_Case_Surveillance_Public_Use_Data.csv")

IOS_country_code = pd.read_csv(r"/kaggle/input/iso-country-codes-global/wikipedia-iso-country-codes.csv")

# DS consolidation

In [ ]:
# Only include confirm cases
toronto = toronto[toronto.Classification == "CONFIRMED"]

hong_kong


In [ ]:
# Only include confirm cases
hong_kong = hong_kong[hong_kong.Confirmed == "Confirmed"]

Clean.clean_by_comparison(hong_kong, "HK_Non_HK_resident", "HK_Non_HK_resident",
["Unknown" , "Non-HK resident","Non-HK Resident" ], np.nan)

# Remove the cases that belonging to China
hong_kong = hong_kong[hong_kong.HK_Non_HK_resident.notnull()]

canada

In [ ]:
# Consolidation of information from canada_dead to canada_cases
indexs = canada_dead.index[canada_dead.case_id.notnull()]
canada_cases['deceased_date'] = np.nan

for read_indx in indexs:   
    case_id_dead = canada_dead.loc[read_indx, "case_id"]
    write_indx = canada_cases.index[canada_cases.case_id == case_id_dead ]
    canada_cases.loc[write_indx, 'deceased_date'] = canada_dead.loc[read_indx, 'date_death_report']

In [ ]:
# duplicated  exsam
p = [ 'age', 'sex', 'health_region','province', 'country', 
     'date_report', 'report_week', 'travel_yn','travel_history_country', 'locally_acquired', 
     'case_source','additional_info', 'additional_source', 'method_note']

o = [ 'age', 'sex','health_region', 'province', 'country', 'date_death_report',
       'death_source', 'additional_info', 'additional_source']

print(len(canada_cases.index[canada_cases.duplicated()== True]))

canada = canada_cases
canada

In [ ]:
# RAM save
del canada_dead, o,p

usa

In [ ]:
usa = usa[usa.current_status == "Laboratory-confirmed case"]

mexico

In [ ]:
# Only include confirm cases
mexico = mexico[mexico["RESULTADO"] == 1]

datasets list

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world, kerla, hong_kong, toronto, usa]
datasets_name_all = ["france", "tunisia", "japan", "indonesia", "korea", "new_zealand", "canada", "guatemala" , "singapore", 
                 "philippines", "india_data", "india_wiki", "vietnam", "colombia", "mexico","world", "kerla", "hong_kong", "toronto", "usa"]

# datasets.shape

In [ ]:
o = []
for i in range(len(datasets_all)):
    print(datasets_name_all[i],datasets_all[i].shape)
    o.append(datasets_all[i].shape[0])
print("\nnum of i " + str(sum(o)))

# Datasets columns

In [ ]:
garbge = [print("\n"+datasets_name_all[i], [i for i in datasets_all[i].columns]) for i in range(len(datasets_name_all))]

# change name of columns

In [ ]:
%%time
france.rename(columns={"health":"severity_illness","status":"treatment"}
              , inplace = True)

tunisia.rename(columns={"date":"confirmed_date", "gender":"sex", "situation":"severity_illness", 
                        "return_from":"infection_place", "health":"background_diseases_origin", "hospital_name":"treatment"},
                           inplace = True)

japan.rename(columns={"No.":"id", "Fixed date":"confirmed_date","Age":"age_band", "residence":"region",
                      "Surrounding patients *":"infected_by"}, inplace = True)

indonesia.rename(columns={"patient_id":"id","gender": "sex", "province":"region", "hospital":"treatment",
                          "contacted_with":"infected_by", "current_state":"severity_illness"}, inplace = True)

korea.rename(columns={"patient_id":"id", "disease":"more_data_and_background_diseases_binary",
                      "state":"severity_illness",
                      "province":"region", "infection_case" :"infection_place",
                      "symptom_onset_date":"date_onset_symptoms", "age": "age_band"}, inplace = True)

world = world.rename(columns={ "province":"region", "date_confirmation": "confirmed_date",
                              "chronic_disease_binary":"more_data_and_background_diseases_binary", 
                              "chronic_disease":"background_diseases_origin",
                              "outcome":"origin_severity_illness", 
                              "travel_history_location":"infection_place",'travel_history_dates': "return_date",
                              "symptoms": "symptoms_origin"})

new_zealand.rename(columns={'Location':'city', 'Age Band':"age_band", 'Gender': "sex", 
                          "Detection Date": "confirmed_date", 'Arrival Date':"return_date",
                          'Origin of Travel (Country)':"infection_place", }, inplace = True)

guatemala.rename(columns={'department': 'region', 'illness':"background_diseases", 'arrival_date':"return_date", 
                          'infection_cause': "infection_place",'symptom_start_date':"date_onset_symptoms",
                          'confirmation_date':'confirmed_date','recovery_date':'released_date'},
                             inplace = True)

singapore.rename(columns={"case": "id", 'date':'confirmed_date', 'gender':"sex", 'hospital':"treatment", 
                          'imported':'infection_place', 'link':"infected_by",
                          'date discharged':'released_date'}, inplace = True)

philippines.rename(columns={'Sex':"sex", 'Age': 'age','Residence in the Philippines':'city', 
                            "Travel History":"infection_place", 'Epi_Link':"infected_by", 
                            "Date of Onset of Symptoms":"date_onset_symptoms", 'travel_history_dates': "return_date", 
                            "Date of Lab Confirmation":'confirmed_date', "Admission / Consultation":"treatment",
                            'Date of Admission':"date_admission_hospital", "Symptoms": "symptoms_origin", 
                            'Other disease':"background_diseases_origin", 'Health Status':"severity_illness", 
                            "Date of Final Status (recovered/expired)":'date_death_or_discharge'}, inplace = True)


india_data.rename(columns={"detected_city":'city', 'detected_state':'region', 'diagnosed_date':'confirmed_date', 
                           'gender':"sex", 'current_status':"severity_illness", 
                           "notes":'infection_place', 'contacts':"link_sicks", 
                           'status_change_date':'date_death_or_discharge',
                           'deceased':'deceased_date', 'link_sicks':"infected_by"}, inplace = True)

india_wiki.rename(columns={'Patient Number':'id', 'Date Confirmed':'confirmed_date', 'Status':"severity_illness", 
                           'gender':"sex", 'current_status':"severity_illness", 'Contact With':"infected_by", 'Outcome Date':'date_death_or_discharge',
                           "Patient Location":'city','Sex':"sex", 'Age': 'age','Comments':"link_sicks"}, inplace = True)


canada.rename(columns={'province':'region', 'date_report':'confirmed_date',
                       'travel_history_country':"infection_place",
                       'current_status':"severity_illness", 
                        'locally_acquired':"infection_cause", "Patient Location":'city',
                        'Sex':"sex", 'Age': 'age_band','Comments':"link_sicks"}, inplace = True)

vietnam.rename(columns={"ID":"id","Gender": "sex", 'Age': "age", 'Detection Location':"region", "Hospital":"treatment",
                        'Confirmed Date': "confirmed_date", 
                        'Travel Country, Correct':"infection_place","Source of Infection":"infected_by", 
                        'Detail Symptoms When Confirmed (clean up)': "symptoms_origin", 
                        'Flight ID (Date)':"return_date", 
                        'Underlying Health Condition': "background_diseases_origin", 'Discharged Date': "released_date",
                        'Critical/severe condition':"severity_illness"}, inplace = True)

mexico.rename(columns={'SEXO': "sex", 'EDAD': "age",
                        'TIPO_PACIENTE':"treatment",'FECHA_INGRESO': "confirmed_date", 
                        'FECHA_SINTOMAS': "date_onset_symptoms", 'FECHA_DEF': "deceased_date"}, inplace = True)

colombia.rename(columns={"id": 'ID DE CASO',
                         'SEXO': "sex", 'EDAD': "age", 
                         'NOMBRE DEPARTAMENTO':"region",
                         'NOMBRE MUNICIPIO': "city",
#                          "CODIGO ISO DEL PAIS" : "country", ???
                         'UBICACION DEL CASO':"treatment_origin",
#                          'FECHA DE DIAGNOSTICO': "confirmed_date",
#                          'FECHA DE INICIO DE SINTOMAS': "date_onset_symptoms",
#                         'FECHA RECUPERADO': "released_date"
#                          ,'FECHA DE MUERTE': "deceased_date",
                        "ESTADO":"severity_illness",
                         'TIPO DE CONTAGIO': "infection_place", 
                        "TIPO DE RECUPERACION": "recovery_test"}, inplace = True)


kerla.rename(columns={'Gender': "sex", 'Age Bracket': "age", "Origin Country": "country",
                         'Notes':"infection_place" , 'Related Patients': "infected_by",
                         'Date Announced': "confirmed_date", 
                         'Released': "released_date",'Deceased': "deceased_date",
                        'Status':"severity_illness", 'Residence City, District':"city"}, inplace = True)

hong_kong.rename(columns={'Gender': "sex", 'Age':"age", 'Date_of_onset': "date_onset_symptoms",
                          'Hospitalised_Discharged_Decease':"severity_illness" , 
                          'Date_of_laboratory_confirmation': "confirmed_date",
                         'District':"region"}, inplace = True)


toronto.rename(columns={'Client Gender': "sex", "Age Group": "age_band", 'Episode Date': "date_onset_symptoms",
                          'Outcome':"severity_illness" ,"treatment":"Ever Hospitalized" }, inplace = True)

usa.rename(columns={ "onset_dt": "date_onset_symptoms", 
                    'age_group': "age_band",
                   }, inplace = True)

# drop feature

# drop

In [ ]:
%%time
france  = france.drop([ 'group', 'infection_reason', 'infection_order', 'contact_number', 'source', 'comments'],axis=1)

indonesia = indonesia.drop(['nationality'],axis=1)

japan = japan.drop(["Close contact situation"],axis=1)

korea = korea.drop(["contact_number"],axis=1)

new_zealand = new_zealand.drop(['MOH Case Number',"Contraction in NZ", 'Outside of Infection Period', 'Origin of Travel (City)',
                                'Travel Destination in NZ', 'Cities visited in Transit', 'Flight Numbers'],axis=1)
guatemala = guatemala.drop(['birth_date','group','source'],axis=1)

singapore = singapore.drop(['nationality', 'cluster_local'],axis=1)

philippines = philippines.drop(['Nationality','Date of Announcement to the Public',"Status", 'Latitude', 'Longitude',
                                'Residence Lat', 'Residence Long','Age Group','Location'],axis=1)

india_data = india_data.drop(["id",'government_id','nationality', 'detected_district', 'current_location', 
                              'current_location_pt', 'detected_city_pt'],axis=1)

india_wiki = india_wiki.drop(['Origin Location'],axis=1)

canada =  canada.drop(['case_id', 'provincial_case_id','health_region', 'report_week', 'travel_yn',
                       'case_source','additional_source', 'method_note', "additional_info" ],axis=1)

vietnam =  vietnam.drop(['Nationality', 'Treatment Location', 'Health Condition When Confirmed', 
                       'Detail Symptoms When Confirmed','Travel History',  'Reference 1', 'Reference 2', 
                       'Reference 3', 'Reference 4', 'Reference 5', 'Travel Country', 'Relationship', 'Infection Cluster'],
                        axis=1)

mexico =  mexico.drop( ['Unnamed: 0','SECTOR', 'ENTIDAD_UM', 
                        'NACIONALIDAD', 'ENTIDAD_RES'],
                        axis=1)

colombia =  colombia.drop( ['FECHA REPORTE WEB', 'FECHA DE NOTIFICACION',
    'CODIGO DIVIPOLA DEPARTAMENTO','CODIGO DIVIPOLA MUNICIPIO',  
    'FECHA REPORTE WEB DIA', 'FECHA REPORTE WEB MES',
    'FECHA REPORTE WEB ANIO', 'FECHA REPORTE WEB NOMBRE MES',
    'FECHA REPORTE WEB DIA SEMANA', 'FECHA DE NOTIFICACION DIA', 
    'FECHA DE NOTIFICACION MES', 'FECHA DE NOTIFICACION ANIO',
    'FECHA DE NOTIFICACION NOMBRE MES', 'FECHA DE NOTIFICACION DIA SEMANA',
    'FECHA DE INICIO DE SINTOMAS DIA', 'FECHA DE INICIO DE SINTOMAS DIA SEMANA', 
    'FECHA DE MUERTE DIA','PERTENENCIA ETNICA', 'NOMBRE DEL GRUPO ETNICO',
    'FECHA DE MUERTE MES', 'FECHA DE MUERTE ANIO',
    'FECHA DE MUERTE NOMBRE MES', 'FECHA DE INICIO DE SINTOMAS MES',
    'FECHA DE INICIO DE SINTOMAS ANIO',
    'FECHA DE INICIO DE SINTOMAS NOMBRE MES', 'FECHA DE MUERTE DIA SEMANA',
    'FECHA DE DIAGNOSTICO DIA', 'FECHA DE DIAGNOSTICO MES',
    'FECHA DE DIAGNOSTICO ANIO', 'FECHA DE DIAGNOSTICO NOMBRE MES',
    'FECHA DE DIAGNOSTICO DIA SEMANA', 'FECHA DE RECUPERACION DIA',
    'FECHA DE RECUPERACION MES', 'FECHA DE RECUPERACION ANIO',
    'FECHA DE RECUPERACION NOMBRE MES','NOMBRE DEL GRUPO ETNICO',
    'FECHA DE RECUPERACION DIA SEMANA'],axis=1)


world = world.drop(['latitude', 'longitude', 'geo_resolution', 
                    'date_admission_hospital','lives_in_Wuhan', 'infection_place', 
                    'reported_market_exposure', 'additional_information','source', 'sequence_available', 
                    'notes_for_discussion', 'location', 'admin3', 'admin2', 'admin1', 
                    'country_new', 'admin_id', 'data_moderator_initials', 'travel_history_binary'],axis=1)




kerla = kerla.drop(['1', 'Unnamed: 3', 'Date Added', 'Detected City', 'Detected District',
                    'Transmission Type',  'Known Cluster', 'DHS (Orig) Patient Number', 'Origin State', 
                    'District Patient Number', 'City Patient Number','Recovery Time', 'Source(s)'],axis=1)


hong_kong = hong_kong.drop(['OBJECTID',  'GlobalID','Case_no_', "個案編號", 
                            'Name_of_hospital_admitted', "發病日期", 
                            'Confirmed',"Shape__Area", 'Shape__Length', 'Status_Chi', '地區', 
                            '性別', '年齡', '入住醫院名稱', '住院_出院_死亡', '香港_非香港居民', 
                            '個案分類', '確定', 'Status', 'Case_classification',"實驗室確診報告日期", 'HK_Non_HK_resident'],axis=1)


toronto = toronto.drop([ 'Currently Intubated', 'Currently in ICU', 'Currently Hospitalized'
                         ,'Reported Date','_id', 'Outbreak Associated','Neighbourhood Name',
                         'FSA','Source of Infection', 'Classification' ],axis=1)

usa = usa.drop([ 'Race and ethnicity (combined)', 'cdc_report_dt' ],axis=1)

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world, kerla, hong_kong, toronto, usa]
datasets_name_all = ["france", "tunisia", "japan", "indonesia", "korea", "new_zealand", "canada", "guatemala" , "singapore", 
                 "philippines", "india_data", "india_wiki", "vietnam", "colombia", "mexico","world", "kerla", "hong_kong", "toronto", "usa"]

In [ ]:
garbge = [print("\n"+datasets_name_all[i], [i for i in datasets_all[i].columns]) for i in range(len(datasets_name_all))]

# format col

times : date_onset_symptoms, confirmed_datedeceased_date, released_date, return_date

In [ ]:
%%time
# Manual care - handel on all 
# 1.61 s

# most havy section 5 => 0.893

t1 = []
t1.append(time.time())  

# japan
japan["confirmed_date"]  =japan["confirmed_date"].apply(lambda x: np.nan if x=='investigating' else x)
print(1)
t1.append(time.time())  

# philippines 
philippines.loc[ philippines.date_onset_symptoms == 'Asymptomatic', "symptoms_origin"] ="no_symptom"

Clean.clean_by_comparison(philippines, "confirmed_date", "confirmed_date",
                          [ 'For Validation', 'For validation'], np.nan)

Clean.clean_by_comparison(philippines, "date_onset_symptoms", "date_onset_symptoms",
                          ['Asymptomatic', 'For Validation', 'For validation'], np.nan)
print(2)
t1.append(time.time()) 

# kerla 
Clean.clean_by_comparison(kerla, "deceased_date", "deceased_date",
                          ["16"], np.nan)

# tunisia
tunisia["return_date"] = tunisia["return_date"][tunisia["return_date"] == 'Local'].apply(lambda x:np.nan)
print(3)
t1.append(time.time()) 

# vietnam
for indx in vietnam.index[vietnam.released_date.notnull()]:
    i = vietnam.released_date[indx]
    if "(" in i:
        vietnam.loc[indx, "released_date"] = np.nan
t1.append(time.time()) 

# vietnam : return_date
indx_full = []
for indx in vietnam.index[vietnam.return_date.notnull()]:
    y = Clean.clean_str_replace(vietnam.return_date[indx], ["(",")"], " ")
    y = y.split(" ")
    
    t = [x for x in y if "/" in x]
    if len(t)>0:
        vietnam.loc[indx, "return_date"] = t[0]
        indx_full.append(indx)

l = list(vietnam.index[vietnam.return_date.notnull()].difference(indx_full))
l.append([236])
for ind in l :
    vietnam.loc[ind, "return_date"] = np.nan
print(4)
t1.append(time.time()) 

# colombia
# for col in ["released_date", "deceased_date", "date_onset_symptoms", "confirmed_date"]:
#     colombia[col] = colombia[col].apply(lambda x: np.nan if x== "-" else x)
print(5)
t1.append(time.time()) 


# mexico
mexico["deceased_date"] =mexico["deceased_date"] .apply(lambda x: np.nan if x== "9999-99-99" else x)
print(6)
t1.append(time.time()) 

# world
index_world = world.index[world["return_date"] =='06.01.2020, 11.01.2020, 17.01.2020']
Clean.update_by_index(world, "return_date",index_world , '17.01.2020')

index_world = world.index[world["return_date"] =='21.01.2020 - 23.012020']
Clean.update_by_index(world, "return_date",index_world , '21.01.2020 - 23.01.2020')

index_world = world.index[world["date_onset_symptoms"] =="08.03.20202"]
Clean.update_by_index(world, "date_onset_symptoms",index_world , "08.03.2020")
print(7)

#  hong_kong 
hong_kong.loc[ hong_kong.date_onset_symptoms == 'Asymptomatic', "symptoms"] ="no_symptom"
Clean.clean_by_comparison(hong_kong , "date_onset_symptoms", "date_onset_symptoms",
                          ['Asymptomatic', 'Unknown',"Mid- July", "Pending", 'Mid-March', "January"], np.nan)
t1.append(time.time()) 

In [ ]:
%%time
# user user 38.7 s


t = []
t.append(time.time()) 
# -confirmed_date-
CTime.updat_s_time_basic_multi_df([france, korea, indonesia, tunisia, japan, canada,
                             singapore, guatemala, philippines,india_wiki, india_data,
                            mexico, kerla, hong_kong],"confirmed_date", False) # colombia,
t.append(time.time())
print(7.5)

CTime.updat_s_time_basic_multi_df([vietnam],"confirmed_date", "%m/%d/%y")
t.append(time.time()) 
print(8)

# -deceased_date-
CTime.updat_s_time_basic_multi_df([france, indonesia, korea,  
                                   canada, guatemala, 
                                    mexico, kerla],
                                  "deceased_date", False) # colombia,
t.append(time.time())
print(9)

# -released_date-
CTime.updat_s_time_basic_multi_df([france, indonesia, korea, singapore, guatemala,
                                   kerla], "released_date", False)  # colombia,
CTime.updat_s_time_basic_multi_df([vietnam],"released_date", "%m/%d/%y")
t.append(time.time()) 
print(10)

# -return_date-
CTime.updat_s_time_basic_multi_df([tunisia, new_zealand, guatemala], "return_date", False)
CTime.updat_s_time_basic_multi_df([vietnam],"return_date", "%m/%d/%Y")
t.append(time.time()) 
print(11)

# -date_onset_symptoms-
CTime.updat_s_time_basic_multi_df([ korea, philippines,
                                   mexico, guatemala, hong_kong, 
                                   toronto ], 
                                  "date_onset_symptoms", False)# colombia, 

CTime.updat_s_time_basic_multi_df([usa],"date_onset_symptoms", "%Y/%m/%d")
t.append(time.time()) 
print(12)

# -date_death_or_discharge-
CTime.updat_s_time_basic_multi_df([ philippines, india_data, india_wiki, world], 'date_death_or_discharge', False)
t.append(time.time()) 
print(13)


for j in ["confirmed_date", "return_date", "date_onset_symptoms"]:
    error = CTime.updat_s_time(world, j, "-", earliest=True)
    print(error)
    
t.append(time.time()) 

In [ ]:
for x in range(0,8):
    print(t1[x+1]-t1[x],x)

In [ ]:
for x in range(0,8):
    print(t[x+1]-t[x], x+7)

sex

In [ ]:
datasets = [france, tunisia, japan, new_zealand, canada, guatemala, singapore, 
            philippines, india_data, india_wiki, vietnam, mexico, colombia, kerla, hong_kong, toronto, usa]

sex_dict = {"female":["Woman", "F", "Female", "f", "FEMALE", 2],
            "male":["Man", "M", "male\xa0?", "Male", "m", "MALE", 1],
            "transgender": ["TRANSGENDER"],
            np.nan: ["Checking", "investigating", "Not Reported", "x", "Unknown",
                     "Unspecified", "UNKNOWN","OTHER", "Missing", "Other" ]}

for dataset in datasets:
            dataset["sex"] = dataset["sex"].apply(lambda x: Pexpansion.get_key_from_dict_by_value(sex_dict,x))

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world , kerla, hong_kong, toronto, usa]
Control.examining_values_by_col (datasets_all , datasets_name_all , "sex") 

# Complete features

symptoms

In [ ]:
%%time
# 1min 7s
symptoms_bag_words = {"pneumonia":["pneumonia","pneumon"],
                        "hypertension" : ['hypertens'],
                        "fever":["fever", 'febr' , 'feverish'], 
                        "cough": ["cough", 'dcough', 'toss', 'couh'],
                        "fatigue":["fatigu", 'fatigur','tired', 'lethargi', 'tire'],
                        "discomfort": ["discomfort", 'unwel'],
                        "weakness": ["weak", 'slump', 'somnol'],
                        "dizziness": ["dizzi"],
                        "rhinorrhoea": ["rhinorrhoea", 'rhinorrhea', 'cold'],
                        "sneezing": ["sneez"],
                        "sputum": ["sputum", "expector", 'phlegm', 'mucus'],
                        "headache": ["headach"],
                        "backache" :['backach'],
                        "diarrhea": ["diarrhea", 'diarrhoea'],
                        "chills": ["chill"],
                        "breathing difficulty": ["dyspnea", "dsypnea"],
                        "rigor": ["rigor"],
                        "pharyngalgia": ["pharyngalgia", 'pharyng', 'pharynx'],
                        "no_symptom": ["asymptomat", 'oligosymptomat', 'asymptomatic?'],
                        "nausea": ["nausea"],
                        "myalgia":["malais", "myalgia", 'milagia', 'mialgia'],
                        'anorexia':['anorexia', 'inapat', 'inappet'],
                        "conjunctivitis" : ['conjunct'],
                        "toothache" :['toothach'],
                        "vomiting": ['vomit', 'emesi'],
                        "sweating":['sweat'],
                        "wheezing": ['wheez'], 
                        'gasp': ['gasp'],
                        "dysphagia":['dysphagia'],
                        "sepsis": ['sepsi'],
                        "gastritis": ['gastriti'],
                        "hypoxia" : ['hypoxia'],
                        "cardiac arrhythmia": ['arrhythmia'],
#                       "ECMO": ['ecmo'],
                        'azotemia':['azotemia'],
                     'influenza':['influenza'], 
                     "rash": ["rash"], 
                     "atherosclerosis": ["atherosclerot",'atherosclerosi'],
                     "diabetes": ["diabeti"]}

symptoms_sentences_bag = {"fever":['flu', 'like'],
                            "nasal congestion":['nasal', 'congest', 'discharg','flu', 'like'],
                            "pleural effusion":['pleural', 'effus'],
                            "sore throat":['sore', "throat",'flu', 'like', 'throath', 'throa'], 
                            "pleuritic chest pain": ['pleurit', 'chest', 'pain', 'pressur'],
                            "chest distress":['chest', 'distress', 'tight'],
                            "muscular stiffness":['muscular', 'stiff'],
                            "myalgia":[ 'general', 'muscular',"sore", 'ach','muscl', "pain",'bodi', 'limb', 'flu', 'like'],
                            "joint pain":[ 'stiff','joint', 'pain'],
                            "rhinorrhoea": ['runni', 'nose','run'],
                            "stuffy nose": ['stuffi', 'nose'],
                            "bleeding nose":['bleed', 'nose'],
                            "weakness": ['lack', 'energi','flu', 'like'],
                            "heart failure":['heart', 'failur'],
                            "breathing difficulty":['difficulti', 'breath','short', 'difficuti', 'distress','respiratori',
                                                    'troubl', 'problem', 'sintoma', 'respirat', 'symptom', 'complaint'],
                            "esophageal reflux":['esophag', 'reflux'],
                            "abdominal pain":['abdomin', 'pain'],
                            "respiratory infection":['pulmonari', 'inflamm', 'respiratori', 'infect','(arvi)', 'diseas'],
                            "bone pain": ['bone', 'pain'],
                            "kidney failure":['kidney', 'failur','renal', 'injuri'],
                            "aggressive pulmonary":['aggress', 'pulmonari'], 
                            "no_symptom": ['no', 'symptom'],
                            "eye irritation": ['eye', 'irrit'],
                            "lesions on chest": ['lesion',  'chest'],
                            "dry mouth":['dri', 'mouth'],
                            "dry throat":['dri', 'throat'],
                            "acute coronary syndrome": ['acut', 'coronari'],
                            "cardiac arrhythmia": ['cardiac', 'arrhythmia', 'dysrhythmia'],
                            "septic shock":['septic', 'shock', 'schock'], 
                            "cardiogenic shock":['cardiogen', 'shock'],
                            "heart attack":['myocardi', 'infarct', 'myocordi'],
                            "electrolyte disorders": ['multipl', 'electrolyt', 'imbal'],
                            "organ failure": ['multipl', 'organ', 'failur'],
                            "primary myelofibrosis":['primari', 'myelofibrosi'],
                            "myocardial dysfunction":['myocardi', 'dysfunct'],
                            "cardiac arrest" :['cardiopulmonari', 'arrest','cardiac', 'arrest'],
                            "respiratory failure":['respiratori', 'failur'],
                            "difficulty walking" :['difficulti', 'walk'],
#                           "oxygen mask": ['oxygen', 'mask', "oxi"],
#                           "dialysis" : ['dialysi'],
#                           "mechanical ventilation":['mechan', 'ventil'],
                            "gastrointestinal bleeding":['gastrointestin', 'bleed'],
                          "multisystem inflammatory syndrome to covid19" : ["syndrom" ,"secondari" ,"covid"]


}

for df in [ philippines, vietnam, world]:
    TextAnalysis.analyze_text_for_multicategory_col(df,  "symptoms_origin" , "symptoms", symptoms_bag_words, symptoms_sentences_bag)

    
TextAnalysis.analyze_text_for_multicategory_col(philippines,  "Final Diagnosis" , "diagnosis", symptoms_bag_words, symptoms_sentences_bag)

In [ ]:
Clean.unite_multicategries_columns(philippines,[ "symptoms", "diagnosis"])

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world , kerla]

Control.examining_values_by_col(datasets_all , datasets_name_all , "symptoms") 

background_diseases

In [ ]:
%%time
# 17.2 s
background_diseases_bag_words= {"hypertension":["hypertens",'hypertenst'],
                                "tuberculosis":['tuberculosi'],
                                'myeloma':['myeloma'],
                                'pneumonia': ['pneumonia'],
                                "diabetes":['diabet', "mellitus"],
                                "tuberculosis": ["tuberculosi"],
                                "atherosclerosis":['atherosclerosi'],
                                "parkinson": ["parkinso", "madopar", 'parkinson'],
                                "stenocardia" : ['stenocardia'],
                                "hypertriglyceridemia": ['hypertriglyceridemia',],
                                "hypothyroidism":['hypothyroid'],
                                "obesity": ['obes',],
                                "dyslipidemia":['dyslipidemia', 'dislipidemia'], 
                                "chronic obstructive pulmonary": ['copd',],
                                "cardiomyopathy":['cardiomyopathi'],
                                "hiv": ["hiv",],
                                "asthma": ["asthma",],
                                "azotemia":["azotemia"],
                                "encephalomalacia": ['encephalomalacia'], 
                                "on dialysis": ["dialysi"],
                                "cardiovascular":["cardiovascular",'cardiac'],
                                "frequent ventricular premature beat":["fvpb"],
                                "stent  implanted" :['stent'],
                               'thalassemia':['thalassemia'],
                               "gastritis":['gastriti'],
                               "leukemia": ['leukemia']}



background_diseases_sentences_bag ={"hypertension":['high', 'blood', 'pressur'],
                                    "prostate cancer":['prostat', 'cancer'],
                                    "colon cancer":['colon', 'cancer'],
                                    "coronary heart disease": ['coronari', 'heart', 'arteri', 'ischem'], 
                                    "coronary bypass surgery": ['bypass', 'surgeri'],
                                    "lung cancer":['lung', 'cancer'],
                                    "tongue cancer":['tongu', 'cancer'],
                                    "valvular heart disease":['valvular', 'heart'],
                                    "renal transplant": ['renal', 'transplant'],
                                    "systemic arterial hypertension":['system', 'hypertens'],
                                    "atrial fibrillation":['atrial', 'fibril'],
                                    "kidney failure":[ 'renal', "insuffici", 'kidney','injuri'],
                                    "chronic kidney disease":['chronic', 'renal' ,'kidney', 'dz', 'diseas'],
                                    "impaired fasting glucose":['impair', 'fast', 'glucos'],
                                    "gastrointestinal bleeding":[ 'git', 'bleed','hemorrhag', 'digest', "tra"],
                                    "arrhythmia":['cardiac', 'dysrhythmia'],
                                    "benign prostatic hyperplasia" : ['benign', 'prostat'],
                                    "chronic obstructive pulmonary disease": [ 'obstruct', 'pulmonari',
                                                                        'condit', 'chronic', 'bronchiti','cronic'],
                                    "prostate hypertrophy": ['prostat', 'hypertrophi'], 
                                    "diabetic nephropathy":['diabet', 'nephropathi'],
                                    "hip replacement": ['hip', "replac"],
                                    "cerebral infarction":['hypertens', 'cerebr', 'infarct'],
                                    "hepatitis B": ["hepat", 'b'],
                                    "frequent ventricular premature beat":["frequent", 'ventricular', 'prematur', 'beat'],
                                    "endocrine condition":['endocrin', 'condit'],
                                    "hypertensive nephrosclerosis":['hypertens', 'nephrosclerosi'],
                                    "peripheral arterial occlusive disease":[ 'arteri', 'occlus' ],
                                   "vestibular disorders": ['vestibular', 'disord'],
                                   "brain haemorrhage":['brain', 'haemorrhag'],
                                    "dilated cardiomyopathy": ["dilat", "cardiomyopathi"],
                                    "prostat cancer": ["prostat", "cancer"]}



for df in [world, philippines, tunisia, vietnam]:
    TextAnalysis.analyze_text_for_multicategory_col(df,  "background_diseases_origin", "background_diseases",
                                                    background_diseases_bag_words , background_diseases_sentences_bag)
                                        

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world ]

Control.examining_values_by_col(datasets_all , datasets_name_all , "background_diseases") 

In [ ]:
%%time
# 1min 12s
# Completing information from other columns
symptoms_to_background_diseases_bag_words ={"diabetes":['diabet', "mellitus"],
                                            "cardiomyopathy":['cardiomyopathi'],
                                            "atherosclerotic": ['atherosclerot'],}

symptoms_to_background_diseases_sentences_bag ={"chronic obstructive pulmonary disease": ['obstruct', 'pulmonari'],
                                                "coronary heart disease": ['ischem', 'heart'],
                                                "chronic kidney disease":['chronic', 'kidney', 'diseas']}


# take data from "symptoms_origin" and "Final Diagnosis"


TextAnalysis.analyze_text_for_multicategory_col(world ,  "symptoms_origin" , "l", symptoms_to_background_diseases_bag_words , symptoms_to_background_diseases_sentences_bag,
                                                input_col_is_dirty=False)

TextAnalysis.analyze_text_for_multicategory_col(philippines,"Final Diagnosis" ,"l", symptoms_to_background_diseases_bag_words , symptoms_to_background_diseases_sentences_bag,
                                                input_col_is_dirty=False)


In [ ]:
#  unite data from different cols 
Clean.unite_multicategries_columns(philippines, [ "background_diseases", "l"])
Clean.unite_multicategries_columns(world, [ "background_diseases", "l"])

In [ ]:
%%time
mexico["background_diseases"] = ""

background_diseases_ls = [" diabetes,", " chronic obstructive pulmonary,"," asthma,", " immunosuppression,"," hypertension,",
                          " cardiovascular,", " obesity,", " kidney failure,"]

cols = ['DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'CARDIOVASCULAR',
        'OBESIDAD', 'RENAL_CRONICA']


for ind in range(len(cols)):
    col = cols[ind]
    name = background_diseases_ls[ind]

    mexico["background_diseases"]  = mexico["background_diseases"] + mexico[col].apply(lambda x: name if x==1 else "")

mexico["background_diseases"]  = mexico["background_diseases"].apply(lambda x: np.nan if x== "" else x)


In [ ]:
# guatemala.background_diseases
background_diseases_cardiopatía =  guatemala.index[guatemala.background_diseases == "cardiopatía"]

Clean.update_by_index(guatemala, "background_diseases" ,background_diseases_cardiopatía ,"coronary heart disease")

background_diseases_nan =  guatemala.index[guatemala.background_diseases == "unknown"].append(guatemala.index[guatemala.background_diseases == "cardiaco"])
Clean.update_by_index(guatemala, "background_diseases" , background_diseases_nan ,np.nan)

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world ]

Control.examining_values_by_col(datasets_all , datasets_name_all , "background_diseases") 

background_diseases_binary

In [ ]:
#TODO VEC if need 
tunisia["background_diseases_binary"] = np.nan
world["background_diseases_binary"] = np.nan

for dataset in [tunisia, world, philippines, guatemala, vietnam, mexico]:
    dataset["background_diseases_binary"] = dataset["background_diseases"].apply(lambda x: 1 if x == x else 0)

#  world   
for dataset in [world]:
    dataset.loc[dataset["more_data_and_background_diseases_binary"]==1, "background_diseases_binary"] = 1

    
world.loc[world.symptoms_origin== "yes", "background_diseases_binary"] = 1

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world ]
Control.examining_values_by_col (datasets_all , datasets_name_all , "background_diseases_binary") 

tretment

In [ ]:
for x in [colombia, world, hong_kong, toronto]:
    x["treatment"] = np.nan


# -del-
philippines.loc[philippines["treatment"] == "For validation", "treatment"] = np.nan
france.loc[france["treatment"] =="deceased", "treatment"] = np.nan
vietnam.loc[vietnam["treatment"] =="?", "treatment"] = np.nan

# notnull == hospitalized
for dataset in [vietnam, singapore, indonesia, philippines]:
    dataset.loc[dataset["treatment"].notnull(), "treatment"] = "hospitalized"
 

#  tunisia  
tunisia.loc[tunisia["hospital_place"].notnull(), "treatment"] = "hospitalized"
tunisia.loc[tunisia["severity_illness"] == "In progress", "treatment"] = "home isolation"
tunisia.loc[tunisia["severity_illness"] == "Stable" , "treatment"] = "hospitalized"
tunisia.loc[tunisia["severity_illness"] == "Critical" , "treatment"] = "hospitalized"
tunisia.loc[tunisia["treatment"] == "Self-insulation", "treatment"] = "home isolation"


#  france 
Clean.clean_by_comparison(france, "treatment", "treatment", ["released","hospital" ], "hospitalized")

# india_data
india_data.loc[india_data["severity_illness"]== "Hospitalized",  "treatment"] = "hospitalized"


# colombia
Clean.clean_by_comparison(colombia,"treatment_origin", "treatment", ["Hospital Uci", "Hospital"], "hospitalized")
colombia.loc[colombia["treatment_origin"] == "Casa", "treatment"] = "home isolation"


# mexico
mexico.loc[mexico["treatment"] == 2, "treatment"] = "hospitalized"
mexico.loc[mexico["treatment"] == 1, "treatment"] = "home isolation"


# kerla
kerla.loc[kerla["severity_illness"] == 'Hospitalized', "treatment"] = "hospitalized"

# hong_kong
hong_kong.loc[hong_kong["severity_illness"] == "Hospitalised",  "treatment"]= "hospitalized"

# toronto
toronto.loc[toronto["Ever in ICU"]== "Yes", "treatment"] = "hospitalized"
toronto.loc[toronto["Ever Intubated"]== "Yes","treatment"] = "hospitalized"

# usa
usa.loc[usa["icu_yn"]== "Yes", 'treatment'] = "hospitalized"
usa.loc[usa["hosp_yn"]== "Yes", 'treatment'] = "hospitalized"
usa.loc[usa["hosp_yn"]== "No", 'treatment'] = "home isolation"

In [ ]:
%%time
# world
bag_words= {"home isolation":["home","quarantin"],
            "clinic":[],
            "hospitalized" :['hospit', "follow-up", 'intub', "fever"],            
            }

sentences_bag = {"home isolation":[ 'not', 'hospit'],
                "clinic":[], 
                "hospitalized" : [ 'from', 'hospit','intens','care','under', 'treatment'],
               }


TextAnalysis.analyze_text_for_category_col(world, 
        "origin_severity_illness", "treatment", bag_words, 
        sentences_bag)


TextAnalysis.analyze_text_for_category_col(world, "symptoms_origin",
            "treatment1", bag_words, sentences_bag, input_col_is_dirty=False)


# concat treatment cols
for col in ["treatment", "treatment1"]:
    world[col] = world[col].apply(lambda x: x if x==x else "")
    world.loc[world[col] == 0, col] = ""
    

dictionary_importance = {"hospitalized":0,"clinic":1,
                         "home isolation":2}

Clean.unite_categories_columns(world, [ "treatment", "treatment1"], dictionary_importance)

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world , kerla, hong_kong, toronto]
Control.examining_values_by_col (datasets_all , datasets_name_all , "treatment") 

severity_illness

In [ ]:
%%time
# format severity_illness to categories (if they give him category col)

severity_illness_dict = {"asymptomatic":["Asymptomatic", "Asintomático"],
            "good":["good","Leve", "Moderado", "Mild", "In progress", "In progress", "isolated"],
            "critical":["critical","Grave","Severe", "Critical"],
            "deceased":["deceased",2,"Fallecido", "Died", "Deceased", "FATAL"],
            "cured":["cured",1,"Discharged", "Recovered", "Cured", "released",
                     "Clinically Recovered", "RESOLVED"],
            np.nan:[0,"Nan", "health", "Hospitalized", "Under Treatment", "To be provided",
                    "Pending admission", "Hospitalised"]}


for dataset in [france, tunisia, indonesia, korea, 
          philippines, india_data,india_wiki,kerla,
             colombia, hong_kong, toronto]:
    dataset["severity_illness"] = dataset.severity_illness.apply(lambda x: 
                               Pexpansion.get_key_from_dict_by_value(severity_illness_dict,x))
    print(0)

In [ ]:
%%time
# Collects information about the severity_illness
# from various columns in the database

for dataset in [canada, guatemala, singapore, vietnam,mexico]:
    dataset["severity_illness"] = ""
    
for dataset in [france, tunisia, indonesia, korea,
                canada, guatemala, singapore, colombia, 
                philippines, india_data,
            india_wiki,vietnam,  mexico ,
                world , kerla, hong_kong, toronto]: 
    dataset["severity_illness2"] = np.nan




#  -- deceased_date and released_date --
datasets_have_both =  [indonesia, france, guatemala , kerla, korea] #  colombia,

# deceased_date
for x in datasets_have_both +[canada, mexico]:
    x.loc[x.deceased_date.notnull(), 'severity_illness2'] = "deceased"
    print(0)

# released_date
for x in datasets_have_both +[singapore, vietnam]:
    x.loc[x.released_date.notnull(), 'severity_illness2'] = "cured"
    print(4)
    
    
 
#  -- Unique --
# singapore - deceased (deceased)
singapore.loc[singapore.deceased == True, 'severity_illness1'] = "deceased"
print(5)


# mexico - critical (UCI)
mexico.loc[mexico.UCI == 1 , 'severity_illness1'] = "critical"
print(6)


# colombia (treatment_origin)
colombia.loc[colombia["treatment_origin"] == "Hospital Uci", 'severity_illness1'] = "critical"
colombia.loc[colombia["treatment_origin"] == "Fallecido", 'severity_illness1'] = "deceased"
colombia.loc[colombia["treatment_origin"] == "Recuperado", 'severity_illness1'] = "cured"
colombia.loc[colombia["treatment_origin"] == "Casa" , 'severity_illness1'] = "good"
colombia.loc[colombia["treatment_origin"] == "Hospital" , 'severity_illness1'] = "good"
# RECUPERADO
colombia.loc[colombia["RECUPERADO"] == "Recuperado" , 'severity_illness2'] = "cured"
colombia.loc[colombia["RECUPERADO"] == "Fallecido" , 'severity_illness2'] = "deceased"
print(7)



# hong_kong
hong_kong.loc[hong_kong["symptoms"] == "no_symptom" ,'severity_illness1'] = "asymptomatic"

# toronto
toronto.loc[toronto.severity_illness == "ACTIVE", 'severity_illness1'] = np.nan
toronto.loc[toronto["Ever in ICU"]== "Yes", 'severity_illness1'] = "critical"
toronto.loc[toronto["Ever Intubated"]== "Yes", 'severity_illness1'] = "critical"

# usa order is matter (critical before deceased)
usa.loc[usa["icu_yn"]== "Yes", 'severity_illness'] = "critical"
usa.loc[usa["death_yn"]== "Yes", 'severity_illness'] = "deceased"

# unit "severity_illness2", "severity_illness1"
for dataset in [toronto, hong_kong,singapore,  mexico, colombia]: 
    Clean.unite_multicategries_columns(dataset, ["severity_illness2", "severity_illness1"])



In [ ]:
%%time
for dataset in datasets_all:
    if "symptoms" in dataset.columns:
         dataset.loc[dataset.symptoms == "no_symptom", "severity_illness5"] ="asymptomatic"
         Clean.unite_multicategries_columns(dataset, ["severity_illness2", "severity_illness5"])

In [ ]:
%%time
# Analyzes text from different columns and adds categories to severity_illness2
bag_words= {"asymptomatic":['asymptomat'],
            "good":["good","stabl", "follow", 'mild',"follow-up", 'moder'],
            "critical":["critic", "intens", "sever", 'unstabl'], 
            "deceased": ["death","dead", "die", "deceas", 'fatal' ],
            "cured":["discharg", "releas", "cure", "recov", 'health','resolv' ],
            "": ["isol", 'valid',  'treatment', "For validation"]}

sentences_bag = {"good":[ 'not', 'hospit', 'in', 'progress' ],
                "critical":['intens','care'], 
                "deceased": [ ],
                "cured":[ "discharged", "hospit" ]}


TextAnalysis.analyze_text_for_category_col(world, "symptoms_origin","severity_illness1", bag_words, sentences_bag,
                             input_col_is_dirty=False)


TextAnalysis.analyze_text_for_category_col(world, "origin_severity_illness","severity_illness", bag_words, sentences_bag,
                             input_col_is_dirty=False)

Clean.unite_multicategries_columns(world, ["severity_illness2", "severity_illness1"] )


In [ ]:
def division_by_symptoms_WHO(p, words_bag):
    # clean str and make a list 
    i = p.split(",")
    i = [x.strip() for x in i]
    
    op = []
    for x in i:
        for k, v in words_bag.items():
            for l in v:
                if l == x:
    
                    op.append(k)

    test_dict = dict(Counter(op))
    if len(test_dict) ==2:
        return "critical"
    elif len(test_dict) == 1:
        return list(test_dict.keys())[0]

In [ ]:
%%time
# add to severity_illness2 severity_illness categories possible  
# by analysis of the symptom column 
# according to the WHO classification (classification by symptoms)

severity_illness_from_symptoms_by_WHO = {"good" :["pneumonia",'fever','cough','fatigue','discomfort','weakness','dizziness','rhinorrhoea','sneezing',
                     'sneezing','sputum','headache','backache','diarrhea','chills','nausea','myalgia',
                     'anorexia','toothache','vomiting','gasp','influenza','sweating ','rigor',
                     'conjunctivitis','pharyngalgia','gastritis','influenza', 'nasal congestion',
                     'sore throat','muscular stiffness','myalgia','joint pain','esophageal reflux',
                     'abdominal pain','bone pain','eye irritation','dry mouth','dry throat',
                     'gastrointestinal bleeding','stuffy nose',"difficulty walking", "pleuritic chest pain"],
     "asymptomatic": ["no_symptom"],
    'critical': ["server" , "breathing difficulty", "dysphagia", "hypoxia", "septic",
                 "cardiac arrhythmia",'organ failure', 'respiratory failure',  'breathing difficulty'
    "sepsis", "hypertension", "azotemia", "pleural effusion","heart failure","kidney failure",
    "chest distress","acute coronary syndrome","cardiac arrhythmia",'septic shock',
    'cardiogenic shock','heart attack','electrolyte disorders','organ failure','primary myelofibrosis',
    'myocardial dysfunction','cardiac arrest','lesions on chest','respiratory failure',"respiratory infection",'primary myelofibrosis']}

dictionary_importance = {"deceased":0,"cured":0, "critical":1, "good":2, "asymptomatic":3}

for dataset in [philippines, world, vietnam]:
    print(0)
    dataset["severity_illness3"] = dataset["symptoms"][dataset["symptoms"].notnull()].apply(lambda x: division_by_symptoms_WHO(x , severity_illness_from_symptoms_by_WHO)) 
    Clean.unite_multicategries_columns(dataset, ["severity_illness2", "severity_illness3"])
    

In [ ]:
# unit cols
for dataset in [france, tunisia,  indonesia, korea,
                canada, guatemala, singapore, philippines, 
                india_data, india_wiki,vietnam, colombia,
                mexico ,world , kerla, hong_kong, toronto]:
    Clean.unite_multicategries_columns(dataset, ["severity_illness", "severity_illness2"])
    dataset["severity_illness_over_time"] = dataset["severity_illness"]
  

In [ ]:
# choose category by dictionary_importance
dictionary_importance = { "deceased":0,"cured":0, "critical":1, "good":2, "asymptomatic":3}

for dataset in [france, tunisia,  indonesia, korea,
                canada, guatemala, singapore, philippines, 
                india_data, india_wiki,vietnam, colombia,
                mexico ,world , kerla, hong_kong, toronto]:
    dataset["t"] =""
    Clean.unite_categories_columns(dataset, ["severity_illness", "t"], dictionary_importance )

In [ ]:
Control.examining_values_by_col (datasets_all , datasets_name_all , "severity_illness_over_time")

In [ ]:
Control.examining_values_by_col (datasets_all , datasets_name_all , "severity_illness")

released_date / deceased_date

In [ ]:
%%time
categories = ["deceased", "cured"]
cols = ["deceased_date","released_date"]

for indx_j in range(len(cols)) :
    j  = cols[indx_j]
    category = categories[indx_j]
    
    for dataset in [philippines, india_data, india_wiki, world]:
        dataset.loc[dataset["severity_illness"] == category,j] = dataset["date_death_or_discharge"]


In [ ]:
Control.examining_values_by_col (datasets_all , datasets_name_all , "released_date") 

age

In [ ]:
%%time
# clean kerla 
kerla["age"] = kerla["age"].astype("str")
Clean.clean_by_comparison(kerla, "age", "age", ["Unspecified", "51F", "nan"], np.nan)


# clean 
dfs_need_super_clean = [world, canada]
for df in dfs_need_super_clean :
    df["age_band"] = df["age"]
    Cage.clean_age_col(df, "age",  ["Not Reported"])

    
# formt to float 
datasets = [india_wiki, india_data,guatemala, philippines, singapore, mexico, colombia, 
            vietnam, kerla, hong_kong, world, canada]
for dataset in datasets:
    dataset["age"] = dataset["age"].astype("float")
    
    
# age from birth_year
france["confirmed_date_year"] = france["confirmed_date"].apply(lambda x: x.year)
france["age"] = france["confirmed_date_year"] - france["birth_year"]
france.drop("confirmed_date_year", axis=1, inplace=True)

In [ ]:
Control.examining_values_by_col (datasets_all , datasets_name_all , "age") 

age_band

In [ ]:
%%time
# Manual care 
# japan
japan.age_band = japan.age_band.apply(lambda x : np.nan if x =="investigating" or x=="Checking" else x )
japan.age_band = japan.age_band.apply(lambda x : "0s" if x =="Under 10" or x=="Under teens" else x )

# canada
Clean.clean_by_comparison(canada, "age_band", "age_band", ["<10","<1"], "1")
print(1)


# usa  20 - 29 Years => 20-29
usa.age_band = usa.age_band.apply(lambda x : np.nan if x =="Unknown"  else x )
usa.age_band = usa.age_band.apply(lambda x : Clean.clean_str_replace(x, [" ", "Years"], "") if x==x else x)
print(2)

# japan.loc[251,"age_band"] = "30s"
# 30s => 30
for df in [korea, japan]:
    df.age_band = df.age_band.apply(lambda x : int(Clean.clean_str_replace(x, ["s"], "")) if x==x else x)
print(3)




# dataset with clean age 
for dataset in [france, tunisia, indonesia, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,france,  kerla, hong_kong]:

     dataset["age_band"] = dataset["age"][dataset["age"].notnull()].apply(lambda x:(x//10)*10)
        
print(4)        

# age_band from dirty age (range and more)
for df in dfs_need_super_clean + [toronto, usa]:
    Cage.to_age_band(df , "age_band",["Not Reported", "19 and younger"])
    df.loc[ df["age_band"] ==  "nan", "age_band"] = np.nan
print(5) 


In [ ]:
# india_data
Control.examining_values_by_col (datasets_all , datasets_name_all ,"age_band") 

country

In [ ]:
%%time
# Complete contry by dataset
datasets = [ france, tunisia, japan ,indonesia, new_zealand, singapore, philippines, india_wiki,
            india_data, colombia, vietnam, mexico, hong_kong, toronto, usa]
datasets_name = [ "france", "tunisia", "japan","indonesia", "new zealand", "singapore",
                 "philippines", "india", "india", "colombia", "vietnam", "mexico", "hongkong", "canada", "usa"]

for indx in range(len(datasets_name)):
    dataset = datasets[indx]
    dataset["country"] = [datasets_name[indx] for i in range(len(dataset))]

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world , kerla, hong_kong, toronto]
Control.examining_values_by_col (datasets_all , datasets_name_all ,"country") 

region

In [ ]:
%%time
india_wiki["region"] = "Karnataka"
kerla["region"] = "kerla"
toronto["region"] ="greatekorea rtorontoarea"
hong_kong["region"] = "hongkong"

city

In [ ]:
%%time
toronto["city"] = "toronto"
Control.examining_values_by_col (datasets_all , datasets_name_all ,"city") 

infected_by

In [ ]:
%%time
# japan
# Manual care
japan.loc[147, "infected_by"]= "No.6"
nan_index = japan.index[japan.infected_by =="investigating"].append([japan.index[japan.infected_by =="None"],
                                                                    japan.index[japan.infected_by =="unknown"]])
Clean.update_by_index(japan, "infected_by", nan_index, np.nan)



# from str ->  index str
for dataset in [japan, singapore]:
    index = dataset.index[dataset.infected_by.notnull()]

    index_e = []
    for indx in index:
        print(dataset.infected_by[indx])
        i = Clean.clean_str_replace(dataset.infected_by[indx],[",", ".","-","No",";","(",")","No."], " " )
        ls = i.split(" ")
        print(ls)
        ls = [str(int(i)-1) for i in ls if i.isdigit()==True]
        dataset.loc[indx, "infected_by"] = Clean.organize_ls_to_str(ls)

In [ ]:
%%time
# from str -> Special id -> index str
for dataset in [france, philippines, vietnam]:
    index = dataset.index[dataset.infected_by.notnull()]

    for indx in index:
        n_ls = [ ]
        i = Clean.clean_str_replace(dataset.infected_by[indx], 
                                     [",", ".","-","No",";","(",")","No."]," ")
        ls = i.split(" ")

        for i in ls:
            [n_ls.append(str(i)) for i in dataset.index[dataset.id == i ].tolist()]
        dataset.loc[indx, "infected_by"] = Clean.organize_ls_to_str(n_ls)


In [ ]:
%%time
# from Special id -> index str
korea.id = korea.id.astype("str")
for dataset in [india_wiki, korea]:
    print("----------------------------------")
    index = dataset.index[dataset.infected_by.notnull()]

    for indx in index:
        i = dataset.infected_by[indx]
        ls = dataset.index[dataset.id == i ].tolist()
        ls = [str(i) for i in ls]
        dataset.loc[indx, "infected_by"] = Clean.organize_ls_to_str(ls)

In [ ]:
def jj(x):
    x = Clean.clean_str_replace(x ,[" ",",","[","]"], " ")
    x = x.split(" ")
    x = [i  for i in x if len(i)> 0 ]
    x = Clean.organize_ls_to_str(x)
    return x

In [ ]:
%%time
india_data["infected_by"] = india_data['link_sicks'][india_data['link_sicks'].notnull()].apply(lambda x: jj(x) )

guatemala.loc[guatemala.infected_by == -99, "infected_by"] = np.nan

In [ ]:
%%time
# kerla
for i in kerla.index[kerla.infected_by.notnull()]:
    if kerla.loc[i ,"infected_by"].isdigit() == False:
        kerla.loc[i ,"infected_by"] = kerla.loc[i ,"infected_by"].replace("P", "")
        kerla.loc[i ,"infected_by"] = kerla.loc[i ,"infected_by"].replace("Kollam", "")
        kerla.loc[i ,"infected_by"] = kerla.loc[i ,"infected_by"].strip()

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world , kerla,  hong_kong, toronto]
datasets_name_all = ["france", "tunisia", "japan", "indonesia", "korea", "new_zealand", "canada", "guatemala" , "singapore", 
                 "philippines", "india_data", "india_wiki", "vietnam", "colombia", "mexico","world", "kerla",  "hong_kong", "toronto"]

Control.examining_values_by_col (datasets_all , datasets_name_all , "infected_by") 

infection_place TODO

In [ ]:
# philippines
# Went to Greenhills 
# France, UAE,
# Korea, USA
# For validation                                    614
# None
# USA
# With Exposure History to a known COVID-19 case
# Diamond Princess Cruise Ship

# singapore
# uk                186
# usa                70
# unknown            36
# uae


# tunisia
# Local
# guatemala
# unknown  
# new_zealand
# UAE
# USA 

# canada
# Not Reported 
# Cruise
colombia.loc[ colombia.infection_place == "Relacionado", "infection_place"] = "colombia"

for i in ["En Estudio", "Importado"]:
    colombia.loc[ colombia.infection_place == i , "infection_place"] = np.nan


recovery_test

In [ ]:
colombia.loc[ colombia.recovery_test == "Tiempo" , "recovery_test"] = "time"
colombia.loc[ colombia.recovery_test == "Pcr" , "recovery_test"] = "pcr test"
colombia.loc[ colombia.recovery_test == "Nan" , "recovery_test"] =  np.nan

In [ ]:
vietnam.loc[ vietnam.severity_illness == "cured" , "recovery_test"] = "pcr test"

smoking

In [ ]:
%%time
mexico["smoking"] = np.nan

mexico["smoking"] = mexico['TABAQUISMO'].apply(lambda x: 0 if x==2 else x )
mexico["smoking"] = mexico["smoking"].apply(lambda x: np.nan if x==98 else x )

In [ ]:
mexico["smoking"].value_counts()

origin

In [ ]:
for indx in range(len(datasets_name_all)):
    dataset = datasets_all[indx]
    dataset["origin"] = datasets_name_all[indx]

In [ ]:
Control.examining_values_by_col (datasets_all , datasets_name_all , "origin") 

# look

In [ ]:
garbge = [print("\n"+datasets_name_all[i], [i for i in datasets_all[i].columns]) for i in range(len(datasets_name_all))]

# build final DS

In [ ]:
datasets_all = [france, tunisia, japan, indonesia, korea, new_zealand, canada, guatemala, singapore, philippines, india_data,
            india_wiki,vietnam, colombia, mexico ,world, kerla ,  hong_kong, toronto, usa]
datasets_name_all = ["france", "tunisia", "japan", "indonesia", "korea", "new_zealand", "canada", "guatemala" , "singapore", 
                 "philippines", "india_data", "india_wiki", "vietnam", "colombia", "mexico","world","kerla",  "hong_kong", "toronto", "usa"]

final_DS = pd.concat(datasets_all, axis=0)

# orgnaze DS

del error row

In [ ]:
d = [indx for indx in final_DS.index if type(indx) == str]

final_DS = final_DS.drop(d, axis= 0)

In [ ]:
final_DS.index = range(len(final_DS))

In [ ]:
lenDS = len(final_DS)
lenDS

In [ ]:
final_DS.columns

# feature 

country

In [ ]:
final_DS.country = final_DS.country.astype("str")
final_DS.country = final_DS.country.apply(lambda x: x.lower())
final_DS.country = final_DS.country.apply(lambda x: x.replace(" ", ""))

country_dict = {"oman":["om"],
            "united kingdom":["uk", "inglaterra", "saintvincentandthegrenadines"],
            "united arab emirates": ["uae"],
            "southkorea": ["corea"],
            "united states":["u.s", "virginislands,u.s."],
            "india":["delhi", "jammuandkashmir"],
               "africa": ["southafrica"],
            np.nan: ["abroad","nan", "unknown"]}


for c in country_dict:
    for y in country_dict[c]:
        final_DS["country"] = final_DS["country"].apply(lambda x: c if x== y else x)

infected_by

In [ ]:
%%time
datasest_with_infected_by = []
for p in range(len(datasets_name_all)):
    d = datasets_all[p]
    c = d.columns
    if "infected_by" in c:
        datasest_with_infected_by.append(datasets_name_all[p])

num_indx = []
for datasest in datasest_with_infected_by:
    num_indx.append(final_DS.index[final_DS.origin == datasest].min()) 
    
p = dict(zip(datasest_with_infected_by,num_indx))
p

In [ ]:
%%time
# clean
def orgnize_id(df, ind):
    

    i = str(df.infected_by[ind])
    u = p[df.origin[ind]]
    print()
    print(i)
    print(df.origin[ind])
    print(u)
    ls = i.split(",")
    i = Clean.remove_from_ls(ls , "")
    ls = [str(float(a)+ u) for a in i]
    i = ",".join(ls)
    print(i)
    return i

In [ ]:
final_DS["id"] =[ ind for ind in range(0,len(final_DS.sex))]
final_DS.id[final_DS["infected_by"].notnull()].apply(lambda ind: orgnize_id(final_DS, ind))

In [ ]:
final_DS["infected_by"].value_counts()

symptoms

In [ ]:
final_DS.loc[final_DS.severity_illness == "asymptomatic" , "symptoms"] = "no_symptom"

In [ ]:
f_all = time.time()

In [ ]:
print(f_all - start_all)

country 

In [ ]:
final_DS.columns

#  Garbage drop 
- Features that have only one dataset or  built with Engineered another feature with them

In [ ]:
[i for i in final_DS.columns]

In [ ]:
c = ['ASMA',
 'CARDIOVASCULAR',
 'CODIGO ISO DEL PAIS',
 'DIABETES',
 'Developing Symptoms',
 'EPOC',
 'Ever Hospitalized',
 'Ever Intubated',
 'Ever in ICU',
 'FECHA DE DIAGNOSTICO',
 'FECHA DE INICIO DE SINTOMAS',
 'FECHA DE MUERTE',
 'FECHA DE RECUPERACION',
 'Final Diagnosis',
 'HIPERTENSION',
 'History of Travel / Exposure',
 'ID',
 'ID DE CASO',
 'INMUSUPR',
 'INTUBADO',
 'NEUMONIA',
 'NOMBRE DEL PAIS',
 'Note',
 'Number of negative test before discharged',
 'OBESIDAD',
 'OTRA_CON',
 'OTRO_CASO',
 'Origin Type',
 'Patient Number',
 'RECUPERADO',
 'RENAL_CRONICA',
 'RESULTADO',
 'Re-Infected',
 'Re-discharged',
 'TABAQUISMO',
 'UCI',
 'UNIDAD DE MEDIDA DE EDAD',
 

 'background_diseases_origin',
 'birth_year',
 'case',
'infection_place',

 'date_admission_hospital',
 'date_death_or_discharge',

 'deceased',

 'departement',
 'hospital_place',
 'id',

 'infection_cause',

 'link_sicks',
 'more_data_and_background_diseases_binary',
 'origin_severity_illness',



 'status',

 'symptoms_origin',

 'treatment_origin',
 'unique_id']



final_DS = final_DS.drop(c, axis= 1)

In [ ]:
# Patient Number, origin_severity_illness
# background_diseases_binary, infected_by, infection_place
# mexico  
for i in final_DS.columns:
    print(i)
    print(final_DS[i].value_counts())
    print("---------------------------------------------------------")

In [ ]:
final_DS = final_DS.drop([], axis=1)

In [ ]:
final_DS.shape

In [ ]:
final_DS.columns

write csv

In [ ]:
%%time
final_DS.to_csv(r'/kaggle/working/Characteristics_Corona_patients - 2 23012021.csv', index = False)
final_DS.to_csv()